In [8]:
from memory.unsafe import DTypePointer
from memory import memset_zero
from memory.buffer import Buffer
from math import sin, atan, cos

alias sixth =1.0/6.0


struct Field[DT: DType, VW: Int]:
    var data: DTypePointer[DT]
    var gx: Int
    var gy: Int
    var gz: Int
    var dsize: Int
    #alias pi = 4.0*atan[DT,1](1.0)
    
    fn __init__(inout self, gx: Int, gy: Int, gz: Int):
        self.dsize = gx * gy * gz
        self.data = DTypePointer[DT].alloc(self.dsize)
        #memset_zero(self.data, self.dsize)
        self.gx = gx
        self.gy = gy
        self.gz = gz

    fn __copyinit__(inout self, other: Self):
        self.gx = other.gx
        self.gy = other.gy
        self.gz = other.gz
        self.data = DTypePointer[DT].alloc(other.dsize)
        self.dsize = other.dsize
        let sd = Buffer[0,DT](self.data, self.dsize)
        let od = Buffer[0,DT](other.data, other.dsize)
        for i in range(0,self.dsize - self.dsize % VW,VW):
            sd.simd_store[VW](i, od.simd_load[VW](i))
        for i in range(self.dsize - self.dsize % VW, self.dsize):
            sd[i] = od[i]

    fn __add__(self, rhs: Field[DT,VW]) raises -> Field[DT,VW]:
        # test if the fields have the same sizes
        if self.gx != rhs.gx or self.gy != rhs.gy or self.gz != rhs.gz :
            raise ("cannot add fields with different grid sizes")
        let result = Field[DT,VW](self.gx, self.gy, self.gz)
        let p : Int
        let igx = self.gx-2  # inner grid size in x dir
        let vwgx = igx - igx%VW # length made of an integer number of vector widths
        for k in range(1,self.gz-1):
            for j in range(1,self.gy-1):
                for i in range (1,vwgx+1,VW):
                    p = k * (self.gx) * (self.gy) + j * (self.gx) + i
                    result.data.simd_store[VW](p, self.data.simd_load[VW](p) + rhs.data.simd_load[VW](p))
                for i in range (vwgx+1,igx+1):
                    p = k * (self.gx) * (self.gy) + j * (self.gx) + i
                    result.data.simd_store[1](p, self.data.simd_load[1](p) + rhs.data.simd_load[1](p))
        return result

    fn __del__(owned self):
        self.data.free()
        print("done del")
        
    fn zero(inout self):
        memset_zero[DT](self.data, self.dsize)

    @always_inline
    fn __getitem__(self, x: Int, y: Int, z: Int) -> SIMD[DT,1]:
        let p = z * (self.gx) * (self.gy) + y * (self.gx) + x 
        return self.data.load(p)

    #@always_inline
    #fn load[nelts:Int=VW](self, x: Int, y: Int, z: Int) -> SIMD[DT,nelts]:
    #    return self.data.simd_load[nelts](z * (self.gx+2) * (self.gy+2) + y * (self.gx+2) + x)

    @always_inline
    fn __setitem__(self, x: Int, y: Int, z: Int, val: SIMD[DT,1]):
        return self.data.store(z * (self.gx) * (self.gy) + y * (self.gx) + x, val)

    #@always_inline
    #fn store[nelts:Int=VW](self, x: Int, y: Int, z: Int, val: SIMD[DT, nelts]):
    #    self.data.simd_store(z * (self.gx+2) * (self.gy+2) + y * (self.gx+2) + x, val)

    # boundaries: all 0 for a start
    fn set_bc(inout self, iis: Int, ie:Int, js:Int, je:Int, ks:Int, ke:Int):
        for k in range(ks,ke):
            for j in range(js,je):
                for i in range(iis,ie):
                   self[i,j,k] = 0.0

    fn initialize(inout self):
        # inner points
        let pi = SIMD[DT,1](4)*atan(SIMD[DT,1](1))  # overkill ?
        for k in range(1,self.gz-1):
            for j in range(1,self.gy-1):
                for i in range(1,self.gx-1):
                    #self[i,j,k] = sin[1, DType.f32](1.0*(i+j+k))
                    self[i,j,k] = sin(pi*i/(self.gx-1)) \
                    * sin(pi*j/(self.gy-1)) \
                    * sin(pi*k/(self.gz-1))
        # bc north
        self.set_bc(0, 1, 1, self.gy-1, 1, self.gz-1)
        # bc south
        self.set_bc(self.gx-1, self.gx, 1, self.gy-1, 1, self.gz-1)
        # bc west
        self.set_bc(1, self.gx-1, 0, 1, 1, self.gz-1)
        # bc east
        self.set_bc(1, self.gx-1, self.gy-1, self.gy, 1, self.gz-1)
        # bc botttom
        self.set_bc(1, self.gx-1, 1, self.gy-1, 0, 1)
        # bc top
        self.set_bc(1, self.gx-1, 1, self.gy-1, self.gz-1, self.gz)

    fn norm2 (self) -> SIMD[DT,1] :
        var s : SIMD[DT, 1] = 0.0
        for k in range(1,self.gz-1):
            for j in range(1,self.gy-1):
                for i in range(1,self.gx-1):
                   # self[i,j,k] = sin[1, DType.f32](1.0*pi)
                   s = s + self[i,j,k] * self[i,j,k]
        return s

In [9]:
#from python import Python
let pi = 4.0*atan[DType.float32,1](1.0)
let n=9
alias w=4
var a = Field[DType.float32,w](n,3,3)
var b = Field[DType.float32,w](n,3,3)
#var c = Field[DType.float32,w](n,1,1)



a.initialize()
print("a norm", a.norm2())
b.initialize()
print("b norm",b.norm2())
for i in range(0,n):
    print(i, a[i,1,1], b[i,1,1], sin((pi*i)/(n-1)))
let c : Field[DType.float32,w] = a + b
print(c.norm2()/a.norm2())
#c.zero()
#for i in range(1,n+1):
#    print(i, c[i,1,1])



a norm 3.9999997615814209
b norm 3.9999997615814209
0 0.0 0.0 0.0
1 0.38268345594406128 0.38268345594406128 0.38268345594406128
2 0.70710676908493042 0.70710676908493042 0.70710676908493042
3 0.92387950420379639 0.92387950420379639 0.92387950420379639
4 1.0 1.0 1.0
5 0.92387950420379639 0.92387950420379639 0.92387950420379639
6 0.70710676908493042 0.70710676908493042 0.70710676908493042
7 0.38268327713012695 0.38268327713012695 0.38268327713012695
8 0.0 0.0 -8.7422776573475858e-08
done del
4.0


In [16]:
var a: SIMD[DType.float64, 4]
var b: SIMD[DType.float64, 4]
var c: SIMD[DType.float64, 4]

a=1.0
b=3.33
c=a+b
print(a,b,c)

[1.0, 1.0, 1.0, 1.0] [3.3300000000000001, 3.3300000000000001, 3.3300000000000001, 3.3300000000000001] [4.3300000000000001, 4.3300000000000001, 4.3300000000000001, 4.3300000000000001]
